# Units in Python

# by Toby Smith
http://tobyrsmith.github.io/

The `Astropy` package includes a powerful framework that allows users to attach **units** to scalars and arrays, and manipulate/combine these, keeping track of the units.

`Astropy` has a lot of built-in units. A complete list can be found [here.](http://docs.astropy.org/en/stable/units/index.html#module-astropy.units.si)

In [1]:
import numpy as np

from astropy.table import QTable

from astropy import units as u
from astropy import constants as const

from astropy.units import imperial
imperial.enable()

*Note: because we imported the `units` package as `u`, you cannot use **u** as a variable name.*

### You can use the units by themselves:

In [2]:
u.m    # The unit of meters

Unit("m")

In [3]:
u.s    # The unit of seconds

Unit("s")

In [4]:
u.m / u.s    # combine them into a composite unit

Unit("m / s")

### For any unit you can find all of the built-in units that are equivalent:

In [5]:
u.m.find_equivalent_units()

  Primary name | Unit definition | Aliases                         
[
  AU           | 1.49598e+11 m   | au, astronomical_unit            ,
  Angstrom     | 1e-10 m         | AA, angstrom                     ,
  cm           | 0.01 m          | centimeter                       ,
  earthRad     | 6.3781e+06 m    | R_earth, Rearth                  ,
  ft           | 0.3048 m        | foot                             ,
  fur          | 201.168 m       | furlong                          ,
  inch         | 0.0254 m        |                                  ,
  jupiterRad   | 7.1492e+07 m    | R_jup, Rjup, R_jupiter, Rjupiter ,
  lyr          | 9.46073e+15 m   | lightyear                        ,
  m            | irreducible     | meter                            ,
  mi           | 1609.34 m       | mile                             ,
  micron       | 1e-06 m         |                                  ,
  mil          | 2.54e-05 m      | thou                             ,
  nmi          | 185

### The `units` package is much more useful when you combine it with scalars or arrays to create `Quantities`

In [6]:
t = 0.25 * u.s

t

<Quantity 0.25 s>

In [7]:
x = np.arange(1,6,1) * u.m    # np.arange(x,y,z) - create an array of numbers between x and y in steps z

x

<Quantity [1., 2., 3., 4., 5.] m>

In [8]:
v = x / t

v

<Quantity [ 4.,  8., 12., 16., 20.] m / s>

### You can access the number and unit part of the Quantity separately:

In [9]:
v.value

array([ 4.,  8., 12., 16., 20.])

In [10]:
v.unit

Unit("m / s")

### At example problem: How long would it take to go 100 km at velocities in `v`?

In [11]:
d = 100 * u.km

T = d / v

T

<Quantity [25.        , 12.5       ,  8.33333333,  6.25      ,  5.        ] km s / m>

### Notice that the units are a bit strange. We can simplify this using `.decompose()`

In [12]:
T.decompose()

<Quantity [25000.        , 12500.        ,  8333.33333333,  6250.        ,
            5000.        ] s>

### Unit conversion is really easy!

In [13]:
v.to(u.cm / u.h)

<Quantity [1440000., 2880000., 4320000., 5760000., 7200000.] cm / h>

In [14]:
v.to(imperial.mi / u.h)

<Quantity [ 8.94774517, 17.89549034, 26.8432355 , 35.79098067, 44.73872584] mi / h>

In [15]:
v.si                     # quick conversion to SI units

<Quantity [ 4.,  8., 12., 16., 20.] m / s>

In [16]:
v.cgs                    # quick conversion to CGS units

<Quantity [ 400.,  800., 1200., 1600., 2000.] cm / s>

In [17]:
p = 3000 * (u.kg / u.m**3)     # From last week's homework

p.to(u.kg / u.km**3)

<Quantity 3.e+12 kg / km3>

### You can define your own units

In [18]:
ringo = u.def_unit('ringo', 3.712 * imperial.yd)

In [19]:
x.to(ringo)

<Quantity [0.29461565, 0.5892313 , 0.88384695, 1.17846261, 1.47307826] ringo>

In [20]:
v.to(ringo / u.s)

<Quantity [1.17846261, 2.35692521, 3.53538782, 4.71385042, 5.89231303] ringo / s>

### Dimentionless Units

In [21]:
Y = (1 * u.m) / (1 * u.km)

Y

<Quantity 1. m / km>

In [22]:
Y.unit

Unit("m / km")

In [23]:
Y.decompose()   # returns the scale of the dimentionless quanity

<Quantity 0.001>

### Some math functions only make sense with dimentionless quanities

In [24]:
np.log(2 * u.m)

UnitTypeError: Can only apply 'log' function to dimensionless quantities

In [25]:
np.log((2 * u.m) / (1 * u.m))

<Quantity 0.69314718>

### Or they expect the correct type of unit!

In [26]:
np.sin(2 * u.m)

UnitTypeError: Can only apply 'sin' function to quantities with angle units

In [27]:
np.sin(30 * u.deg)

<Quantity 0.5>

## Using units can save you headaches. 

* All of the trig functions expect all angles to be in radians. 
* If you forget this, it can lead to problems that are hard to debug

In [28]:
np.sin(90)               # not really what I expected

0.8939966636005579

In [29]:
np.sin(90 * u.deg)       # better

<Quantity 1.>

# Tables and units

Astropy tables are constructed to use units

In [30]:
T = QTable.read('Planets.csv', format='ascii.csv')

In [31]:
T[0:3]

Name,a,col2
str7,float64,float64
Mercury,0.3871,0.2056
Venus,0.7233,0.0068
Earth,0.9991,0.0166


In [32]:
T['a'].unit = u.AU
T[0:3]

Name,a,col2
,AU,
str7,float64,float64
Mercury,0.3871,0.2056
Venus,0.7233,0.0068
Earth,0.9991,0.0166


In [33]:
T['a']

<Quantity [ 0.3871,  0.7233,  0.9991,  1.5237,  5.2016,  9.5424, 19.1727,
           29.9769, 17.8589] AU>

In [34]:
T['a'].to(u.km)

<Quantity [5.79093357e+07, 1.08204140e+08, 1.49463233e+08, 2.27942276e+08,
           7.78148284e+08, 1.42752272e+09, 2.86819510e+09, 4.48448041e+09,
           2.67165341e+09] km>

In [35]:
T['a'].to(u.km)[0]

<Quantity 57909335.74796999 km>

### Tables and units have a few unexpected quirks:

In [36]:
T['a'][0]

<Quantity 0.3871 AU>

In [37]:
T['a'][0].to(u.km)

<Quantity 57909335.74796999 km>

In [38]:
T

Name,a,col2
,AU,
str7,float64,float64
Mercury,0.3871,0.2056
Venus,0.7233,0.0068
Earth,0.9991,0.0166
Mars,1.5237,0.0935
Jupiter,5.2016,0.049
Saturn,9.5424,0.0547
Uranus,19.1727,0.0486
Neptune,29.9769,0.0088


In [39]:
T['a'].value[0]

0.3871

In [40]:
T['a'][0].to(u.km)

<Quantity 57909335.74796999 km>

# Constants

The `Astropy` package also includes a whole bunch of built-in constants to make your life easier.

A complete list of the built-in constants can be found [here.](http://docs.astropy.org/en/stable/constants/index.html#reference-api)

In [41]:
M = 0.1 * u.kg
M / const.M_sun

<Quantity 5.02897844e-32>

In [42]:
const.G

<<class 'astropy.constants.codata2014.CODATA2014'> name='Gravitational constant' value=6.67408e-11 uncertainty=3.1e-15 unit='m3 / (kg s2)' reference='CODATA 2014'>

In [43]:
const.M_sun

<<class 'astropy.constants.iau2015.IAU2015'> name='Solar mass' value=1.9884754153381438e+30 uncertainty=9.236140093538353e+25 unit='kg' reference='IAU 2015 Resolution B 3 + CODATA 2014'>

---

### An Example: The velocity of an object in circular orbit around the Sun is

$$ v=\sqrt{GM_{\odot}\over r} $$

In [44]:
r = T['a']

In [45]:
r[2]

<Quantity 0.9991 AU>

In [46]:
V = np.sqrt(const.G * const.M_sun / r)

V[2]

<Quantity 1.15252761e+10 m(3/2) / (AU(1/2) s)>

In [47]:
V.decompose()[2]

<Quantity 29798.10399489 m / s>

In [48]:
V.to(u.km/u.s)[2]

<Quantity 29.79810399 km / s>

In [49]:
V.to(ringo/u.ms)[2]

<Quantity 8.77898782 ringo / ms>

## Functions and Units - (Last week's homework)

In [50]:
def FindD(H,A):
    result = (1329 / np.sqrt(A)) * (10 ** (-0.2 * H))
    return result * u.km

In [51]:
H = 3.34
A = 0.09

D = FindD(H,A)
D

<Quantity 951.48890004 km>

In [52]:
def FindM(D):
    p = 3000 * (u.kg / u.m**3)
    result = p * (1/6) * np.pi * D**3
    return result

In [53]:
M = FindM(D)
M

<Quantity 1.35310362e+12 kg km3 / m3>

In [54]:
M.decompose()

<Quantity 1.35310362e+21 kg>